In [15]:
import pandas as pd
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.endpoints import PlayerGameLogs
from nba_api.stats.static import players

In [16]:
start_date = "01/20/2025"
end_date = "01/27/2025"
season = "2024-25"

In [17]:
# # Llamar a la API para obtener estadísticas de jugadores en ese período
# player_stats = LeagueDashPlayerStats(
#     season="2024-25",
#     season_type_all_star="Regular Season",  # O "Playoffs" si buscas playoffs
#     date_from_nullable=start_date,
#     date_to_nullable=end_date
# )

# # Convertir los datos en un DataFrame de pandas
# df = player_stats.get_data_frames()[0]

# # Filtrar jugadores que han jugado esa semana (GP > 0)
# df = df[df['GP'] > 0]

# df


In [18]:
box_scores = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
box_scores['GAME_DATE'] = pd.to_datetime(box_scores['GAME_DATE'])

In [19]:
def get_team_records(season):
    # Obtener todos los partidos de la temporada
    gamefinder = LeagueGameFinder(league_id_nullable="00", season_nullable=season)
    games = gamefinder.get_data_frames()[0]
    
    # Convertir las fechas al formato datetime
    games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
    
    # Filtrar los partidos hasta la fecha dada
    filtered_games = games[games['GAME_DATE'] <= pd.to_datetime(end_date)]
    
    # Inicializar diccionario para almacenar victorias y derrotas de cada equipo
    team_records = {}
    
    for team in filtered_games['TEAM_NAME'].unique():
        team_games = filtered_games[filtered_games['TEAM_NAME'] == team]
        wins = team_games[team_games['WL'] == 'W'].shape[0]
        losses = team_games[team_games['WL'] == 'L'].shape[0]
        team_records[team] = {'Wins': wins, 'Losses': losses}
    
    # Convertir a DataFrame
    records_df = pd.DataFrame.from_dict(team_records, orient='index').reset_index()
    records_df.columns = ['Team Name', 'Wins', 'Losses']
    
    return records_df

In [20]:
team_records_df = get_team_records(season)

In [21]:
players_ids = players.get_active_players()
players_name_id_map = pd.DataFrame(players_ids,columns=['id','full_name','first_name'])

In [22]:
box_scores = box_scores.drop(['NICKNAME','DD2','TD3', 'WNBA_FANTASY_PTS','NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK','WNBA_FANTASY_PTS_RANK'],axis=1)

In [ ]:
box_scores.to_excel("./OUTPUT/players_box_scores.xlsx", index=False)
team_records_df.to_excel("./OUTPUT/team_records_to_date.xlsx", index=False)
players_name_id_map.to_excel("./OUTPUT/players_name_id_map.xlsx", index=False)